# MATE testing

In [1]:
%reset -f

#=================================================================================
# Tuning engine (based on tiny genetic programming plus, by moshe sipper)
#=================================================================================

from random import random, randint, seed
from statistics import mean
from copy import deepcopy
from IPython.display import Image, display
from graphviz import Digraph, Source 

from sympy import simplify, symbols

import utils
import CoEvolution
import GP
import GPSetup

#from utils import *
#from CoEvolution import *
#from GP import *
#from GPSetup import *

import numpy as np
import imp
#import platform
#print(platform.system()=='Linux')

imp.reload(utils)
imp.reload(CoEvolution)
imp.reload(GP)
imp.reload(GPSetup)
from utils import *
from CoEvolution import *
from GP import *
from GPSetup import *

# init internal state of random number generator
seed()

In [3]:
from math import inf
import datetime

# GP parameters
POP_SIZE        = 10                   # population size
GEN_POP_SIZE    = round(POP_SIZE*.75)  # size of offspring population
MIN_DEPTH       = 2                    # minimum initial tree depth
MAX_DEPTH       = 4                    # maximum initial tree depth
TOURNAMENT_SIZE = 5                    # size of tournament for tournament selection
XO_RATE         = 0.8                  # crossover rate
PROB_MUTATION   = 0.2                  # per-node mutation probability
BLOAT_CONTROL   = False                # True adds bloat control to fitness function

# Statistical setting
SAMPLE_RUNS        = 1                 # number of samples per GP tree
STAT_SIGNIFICANCE  = 0.05

# Initialisation setting
INIT_ITERATIONS     = 1                # number of repititions for initialisation (searching one parameter and fixing the others)
NB_PARAM_CANDIDATES = 10               # number of parameter values

# maximum number of generations to evolve programs
GENERATIONS     = 10

# Date and time
DT = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

# Debugging
debug = True
init_references()

#==========================================================
# tunner input
#==========================================================
# Algorithm executable
with open('cfg_executable.txt', 'r') as file:
    executable = file.read().replace('\n', '')

# List of instances
with open('cfg_instances.txt', 'r') as file:
    instances = [line.split() for line in file.readlines()[1:]]

# List of parameters
with open('cfg_parameters.txt', 'r') as file:
    parameters = [line.split() for line in file.readlines()[0:]]

print(executable)
print(instances)
print(parameters)


init refs
{'5': -inf, '10': -inf, '20': -inf}
python3 ./input/GA_leadingones.py
[['5', '5'], ['10', '10'], ['20', '20']]
[['mu_rate', '0', '1'], ['x_prob', '0', '1']]


In [4]:
#==========================================================
# Problem description
#==========================================================
print('=============================================')
print('Parameters:')
print('=============================================')
[print(param[0], (': ['+param[1]+','+param[2]+']') if len(param)>=3 else '') for param in parameters]
print('=============================================')

print('=============================================')
print('Features:')
print('=============================================')
print(','.join(FEATURES))
print('=============================================')

print('=============================================')
print('Instances [+Features]:')
print('=============================================')
[print(inst[0]+':', inst[1:]) for inst in instances]
print('=============================================')
#==========================================================

def run_target_wrapper(inst_param):
    return run_target_static(inst_param[0], inst_param[1])

#==========================================================
# Initialise parameter values arbitrarily: 
# use average of parameter interval
#==========================================================
for inst in instances:
    GP.ref_param_values[inst[0]] = []
    for parameter in parameters:
        lbound = float(parameter[1]) if len(parameter)>=3 else -999999999999
        rbound = float(parameter[2]) if len(parameter)>=3 else +999999999999
        GP.ref_param_values[inst[0]].append( str((lbound+rbound)/2.0) )
#print('>>>>>>',GP.ref_param_values)


#==========================================================
# Initialise parameter values based on best score 
# / for each instance
#==========================================================
mean_references = {}  # dict to store mean best parameter values
for inst in instances:
    mean_references[inst[0]] = -inf

for u in range(INIT_ITERATIONS):
    for i in range(len(parameters)):
        # Tune for parameter i
        parameter = parameters[i]

        #==========================================================
        # Generate references fitnesses for each instance
        # and select best
        #==========================================================
        
        param_name = parameter[0]  # parameter to run
        lbound = float(parameter[1]) if len(parameter)>=3 else -999999999999
        rbound = float(parameter[2]) if len(parameter)>=3 else +999999999999
        step = float(rbound-lbound)/NB_PARAM_CANDIDATES
        # calculate initial references
        print("pre-tuning ("+str(u)+")", param_name+":")
        for inst in instances:
            print("instance",inst[0])
            for r in range(0, NB_PARAM_CANDIDATES):
                # generate parameter values
                param_value = lbound + step*r + step/2.0

                tmp = GP.ref_param_values[inst[0]][i]
                GP.ref_param_values[inst[0]][i] = str(param_value)

                #==============================
                # run multiple times
                #==============================
                scores_list = None
                #with Pool(SAMPLE_RUNS) as p:
                with Pool(1) as p:
                    scores_list = p.map(run_target_wrapper, [ [inst[0], GP.ref_param_values[inst[0]] ] ] *SAMPLE_RUNS)
                inst_score = mean(scores_list)
                max_score = max(scores_list)
                #_, pval = stats.ranksums(new_scores, current_scores)

                #print('=> MEAN:', inst_score,'| REF:',mean_references[inst[0]],'|MAX:',max_score)
                
                # if score is better, keep new parameter value and update mean value reference
                if (inst_score > mean_references[inst[0]]):
                    mean_references[inst[0]] = inst_score
                else:
                    GP.ref_param_values[inst[0]][i] = tmp

                # update score references if max current score is better
                if (max_score > GP.references[inst[0]]):
                    GP.references[inst[0]] = inst_score

        print('PARAM REFERENCES | it='+str(u)+' | p='+param_name+':')
        print(GP.ref_param_values)

        print('=============================================')
        print('Score references:')
        print('=============================================')
        for (inst, score) in GP.references.items():
            print(inst+':', score)
        print('=============================================')

Parameters:
mu_rate : [0,1]
x_prob : [0,1]
Features:
n
Instances [+Features]:
5: ['5']
10: ['10']
20: ['20']
pre-tuning (0) mu_rate:
instance 5
instance 10
instance 20
PARAM REFERENCES | it=0 | p=mu_rate:
{'5': ['0.05', '0.5'], '10': ['0.05', '0.5'], '20': ['0.05', '0.5']}
Score references:
5: 5.0
10: 10.0
20: 20.0
pre-tuning (0) x_prob:
instance 5
instance 10
instance 20
PARAM REFERENCES | it=0 | p=x_prob:
{'5': ['0.05', '0.5'], '10': ['0.05', '0.5'], '20': ['0.05', '0.5']}
Score references:
5: 5.0
10: 10.0
20: 20.0


In [5]:
import CoEvolution, GP, GPSetup
imp.reload(utils)
imp.reload(CoEvolution)
imp.reload(GP)
imp.reload(GPSetup)
from utils import *
from CoEvolution import *
from GP import *
from GPSetup import *

#==========================================================
# GP Co-evolution
#==========================================================
pops = CoEvolution()
pops.para_evolution()

'''
#==========================================================
# store final populations
#==========================================================
result_pop = open("./output/result_pop-" + dt + ".txt", "w+")
for i in range(len(parameters)):

    #==========================================================
    # Tune for parameter i
    #==========================================================
    parameter = parameters[i]
    result_pop.write('Parameter: ' + parameter[0] + '\n')
    result_pop.write(str(pops.gp[i].output_csv()) + '\n')
    result_pop.write('==============================' + '\n')
    result_pop.flush()
result_pop.close()
#==========================================================
'''

Co-Evolution:
PARAMETER p_0
Initial population:PARAMETER p_1

Initial population:
i>> i>>10 0.0 1.0
 10 0.0 1.0
i>> 10 0.0 1.0
i>> 10 0.0 1.0


Process Process-61:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-62:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/yaf/Workspace/MATE/mate/CoEvolution.py", line 44, in evolve_param
    self.gp[i].evolution()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yaf/Workspace/MATE/mate/GP.py", line 199, in evolution
    self.init_population()
  File "/home/yaf/Workspace/MATE/mate/GP.py", line 119, in init_population
    self.evaluate(ind)
  File "/home/yaf/Workspace/MATE/mate/GP.py", line 347, in evaluate
    tree.fitness_history = p.map(self.run_target, [tree]*SAMPLE_RUNS)
  File "/usr/lib/python

'\n#==========================================================\n# store final populations\n#==========================================================\nresult_pop = open("./output/result_pop-" + dt + ".txt", "w+")\nfor i in range(len(parameters)):\n\n    #==========================================================\n    # Tune for parameter i\n    #==========================================================\n    parameter = parameters[i]\n    result_pop.write(\'Parameter: \' + parameter[0] + \'\n\')\n    result_pop.write(str(pops.gp[i].output_csv()) + \'\n\')\n    result_pop.write(\'==============================\' + \'\n\')\n    result_pop.flush()\nresult_pop.close()\n#==========================================================\n'